In [1]:
from keras.models import Model, load_model
from keras.layers import Dropout, Conv2D, Cropping2D, Conv2DTranspose, Add, Input, Reshape, Permute, Activation
from keras.applications.vgg16 import VGG16, preprocess_input


/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(480, 480, 3))

for layer in base_model.layers:
    layer.trainable = False

nClasses = 3

block3_pool = base_model.get_layer('block3_pool').output
block4_pool = base_model.get_layer('block4_pool').output
block5_pool = base_model.get_layer('block5_pool').output

conv_name_base = "block%d_conv%d"
pool_name_base = "block%d_pool"
dropout_name_base = "block%d_dropout"

block = 6
step = 1

X = Conv2D(4096, (7, 7),  padding='same', activation='relu', name='block6_conv1')(block5_pool)
X = Dropout(0.5, name='block6_dropout1')(X)
X = Conv2D(4096, (1, 1), activation='relu', padding='same', name='block6_conv2')(X)
X = Dropout(0.5, name='block6_dropout2')(X)
X = Conv2D(nClasses, (1, 1), padding='same', kernel_initializer='he_normal', name='block6_conv3')(X)
X = Conv2DTranspose(nClasses, kernel_size=(4,4), strides=(2,2) , padding='same', use_bias=False, name='block6_deconv1')(X)

X2 = Conv2D(nClasses, (1, 1), kernel_initializer='he_normal', padding='same', name='block7_conv1')(block4_pool)
# X2 = Cropping2D(cropping=((0, 1), (0, 0)))(X2)
X = Add(name='block7_Add')([X, X2])
X = Conv2DTranspose(nClasses, kernel_size=(4,4), strides=(2, 2), padding='same')(X)

X3 = Conv2D(nClasses, (1, 1), padding='same', name='block8_conv1')(block3_pool)
# X3 = Cropping2D(cropping=((0, 3), (0, 0)))(X3)
X = Add(name='block8_Add')([X, X3])
X = Conv2DTranspose(nClasses, kernel_size=(16, 16), strides=(8, 8), padding='same', name='block8_Deconv')(X)

# X = (Reshape((-1, nClasses)))(X)
X = (Activation('softmax'))(X)

model = Model(inputs = [base_model.input], outputs=[X])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 480, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 480, 480, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 480, 480, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 240, 240, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [3]:
model.load_weights('fcn8_vgg16_weights.best.hdf5')

In [4]:
model.save('fcn8_vgg16.model.hdf5')